In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve, auc, make_scorer, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, label_binarize
from data_utils import load_metadata, sample_sounds_by_category, load_wave_data, load_all_sounds,load_paths_from_config
from feature_utils import *

In [ ]:
compute_features=True

In [ ]:
csv_file_path, audio_files_path = load_paths_from_config()


# 1.1) check if the paths exist
assert os.path.exists(csv_file_path), "CSV file not found!"
assert os.path.exists(audio_files_path), "Audio directory not found!"

In [ ]:
# 2) Load metadata
df = load_metadata(csv_file_path)

print(df)
print("Missing values:\n", df.isnull().sum()) # Check for missing values
print(df.describe()) # Show summary statistics

In [ ]:
# 3) Define categories
categories = {
        'Animals': ['dog', 'rooster'],
        'Natural soundscapes & water sounds': ['thunderstorm', 'pouring_water'],
        'Human sounds': ['snoring', 'sneezing'],
        'Interior/domestic sounds': ['clock_alarm', 'vacuum_cleaner'],
        'Exterior/urban noises': ['siren', 'helicopter']
    }

# Filter the dataframe to only include selected categories
selected_classes = sum(categories.values(), [])  # Flatten the dictionary into a list of class names
df_filtered = df[df['category'].isin(selected_classes)]  # Keep only selected categories

class_counts = df_filtered['category'].value_counts()
print("Class counts for selected categories:\n", class_counts)
if all(class_counts == 40):
    print("✅ All classes have 40 rows.")
else:
    print("❌ Some classes do not have 40 rows.")
    print("Classes with insufficient rows:\n", class_counts[class_counts != 40])

# # Plot class distribution of selected categories
# plt.figure(figsize=(10, 5))
# sns.countplot(x=df_filtered['category'])
# plt.xticks(rotation=45)
# plt.title("Class Distribution (Selected Categories)")
# plt.show()

In [ ]:
# Create a mapping dictionary from category to its target number
category_to_target = df_filtered.set_index('category')['target'].to_dict()
mapping_df = pd.DataFrame(category_to_target.items(), columns=['Category', 'Target'])
print(mapping_df.to_string(index=False))

In [ ]:
# 4) Sample sounds
sampled_sounds = sample_sounds_by_category(df, categories)

# Track missing and represented classes
missing_classes = {}
represented_classes = {}

for category, expected_classes in categories.items():
    # Extract actual classes from sampled sounds
    sampled_classes = set(df[df['filename'].isin(sampled_sounds[category])]['category'])
    
    # Check which classes are missing
    missing = set(expected_classes) - sampled_classes
    represented_classes[category] = sampled_classes

    if missing:
        missing_classes[category] = missing

    print(f"Category: {category}, Sampled Classes: {sampled_classes}")

# Summary
print("\n✅ Represented Classes:")
for category, classes in represented_classes.items():
    print(f"- {category}: {classes}")

if missing_classes:
    print("\n❌ Missing Classes:")
    for category, classes in missing_classes.items():
        print(f"- {category}: {classes}")
else:
    print("\n🎉 All expected classes are present!")


In [ ]:
# 5) Load wave data
wave_list_data_sampled = load_wave_data(sampled_sounds, audio_files_path) #load_sample
wave_list_data = load_all_sounds(df, categories, audio_files_path) #load all sounds

print("Total samples loaded:", len(wave_list_data))
print("Example sample rate:", wave_list_data[0][2])
print("Example waveform shape:", wave_list_data[0][3].shape)

In [ ]:
# 7) Extract all features
# keys_list, combined_features = compute_combined_features_for_wave_list(wave_list_data)

In [ ]:
# # 8) Save all features

if compute_features:
    # 7) Extract selected features
    # keys_list, mfcc_list, hist_list, spectral_list, zcr_list, envelope_list, hnr_list = compute_features_for_wave_list(wave_list_data)
    keys_list, mfcc_list, hist_list, spectral_centroid_list, spectral_contrast_list, pitch_features_list, zcr_list, envelope_list, hnr_list = compute_features_for_wave_list(wave_list_data)

    # 8) Save selected features
    save_multiple_features_to_npz(
        keys_list, 
        mfcc_list, 
        hist_list, 
        spectral_centroid_list,
        spectral_contrast_list,
        pitch_features_list, 
        zcr_list, 
        envelope_list, 
        hnr_list, 
        out_file="features/extracted_features_multiple_test.npz"
    )

In [ ]:
# Load saved features
loaded_data = np.load("features/extracted_features_multiple_test.npz")

# Define which features to include (set True to include, False to exclude)
feature_selection = {
    'mfcc': True,
    'hist': True,
    'spectral_centroid': True,
    'spectral_contrast': True,
    'pitch_features': True,
    'zcr': True,
    'envelope': True,
    'hnr': True
}

# Display the shapes of each feature type
print("Feature shapes and selection status:")
for feature, include in feature_selection.items():
    shape = loaded_data[feature].shape
    status = "✅" if include else "❌"
    print(f"{feature.capitalize()} shape: {shape} - Status: {status}")

# Extract features and labels
keys_list = loaded_data['keys']
mfcc_features = loaded_data['mfcc']
hist_features = loaded_data['hist']
spectral_centroid_features = loaded_data['spectral_centroid']
spectral_contrast_features = loaded_data['spectral_contrast']
pitch_features = loaded_data['pitch_features']
zcr_features = loaded_data['zcr']
envelope_features = loaded_data['envelope']
hnr_features = loaded_data['hnr']


# #Display first feature vectors for verification
# print("First MFCC feature vector:", loaded_data['mfcc'][0])
# print("First histogram feature vector:", loaded_data['hist'][0])
# print("First spectral centroid feature vector:", loaded_data['spectral_centroid'][0])
# print("First spectral contrast feature vector:", loaded_data['spectral_contrast'][0])
# print("First pitch feature vector:", loaded_data['pitch_features'][0])
# print("First ZCR feature vector:", loaded_data['zcr'][0])
# print("First Amplitude Envelope feature vector:", loaded_data['envelope'][0])
# print("First HNR feature vector:", loaded_data['hnr'][0])

# Define feature combinations
feature_combinations = {
    'mfcc': mfcc_features,
    'hist': hist_features,
    'spectral_centroid': spectral_centroid_features,
    'spectral_contrast': spectral_contrast_features,
    'pitch': pitch_features,
    'zcr': zcr_features,
    'envelope': envelope_features,
    'hnr': hnr_features
}

# Combine features based on flags
combined_features = combine_features_with_flags(loaded_data, feature_selection)
print("\nCombined features shape:", combined_features.shape)

In [ ]:
### Compare feature distributions for different classes

# Convert combined_features to DataFrame
df_features = pd.DataFrame(combined_features)
df_features["label"] = keys_list  # Add class labels

# Ensure labels are categorical, not just numbers
df_features["label"] = df_features["label"].astype(str)  # Convert to string labels

# Select first 5 features for visualization (modify if needed)
selected_features = df_features.iloc[:, -8:]  
selected_features["label"] = df_features["label"]  # Re-add labels

# Convert to long format for plotting
df_melted = selected_features.melt(id_vars="label", var_name="Feature", value_name="Value")

# --- Strip Plot Fix ---
plt.figure(figsize=(12, 6))
sns.stripplot(x="Feature", y="Value", hue="label", data=df_melted, dodge=True, jitter=True)
plt.title("Feature Distributions with Strip Plot (Fixed Labels)")
plt.xticks(rotation=45)
plt.legend(title="Class Labels", bbox_to_anchor=(1.05, 1), loc='upper left')  # Fix legend
plt.show()



In [ ]:
# Train-Test Splitting
X = combined_features
y = np.array(keys_list)

# Train-test split
X_train_not_scaled, X_test_not_scaled, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=keys_list
)


In [ ]:
# Normalize the data using Z-score normalization (StandardScaler)
scaler = StandardScaler()

# feature normalization:
# feature = feature - mean(feature acrocss all examples) / std(feature across all examples)
## VERY IMPORTANT: the above mean and std must be calcualted from TRAIN data only, not from EVAL or TEST data

# Fit only on the training data and transform both train and test sets
X_train = scaler.fit_transform(X_train_not_scaled)  # Fit on train data only
X_test = scaler.transform(X_test_not_scaled)        # Use train's mean and std to scale test data

In [ ]:
# Print sizes of train and test sets
print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)

# Debugging - Check which classes exist before and after splitting
unique_classes = set(keys_list)
print("Unique classes in dataset before splitting:", unique_classes)

# Count occurrences of each class in train and test sets
train_counts = pd.Series(y_train).value_counts().sort_index()
test_counts = pd.Series(y_test).value_counts().sort_index()

# Print class counts for debugging purposes
print("\nTraining Class Counts:\n", train_counts)
print("\nTest Class Counts:\n", test_counts)

In [ ]:
# Convert lists to Pandas Series for counting
train_counts = pd.Series(y_train).value_counts()
test_counts = pd.Series(y_test).value_counts()

# Merge train and test counts into a single DataFrame
df_counts = pd.DataFrame({'Train': train_counts, 'Test': test_counts}).fillna(0).astype(int)

# Display the counts table with a better format
print("\nClass Distribution Table:")
print(df_counts.to_string())

# Display summary
print("\nSummary:")
print(f"Total samples in Train set: {df_counts['Train'].sum()}")
print(f"Total samples in Test set: {df_counts['Test'].sum()}")


In [ ]:
param_grid_svm = {
    'C': [0.01, 0.1, 1, 10, 100],
    'gamma': [0.01, 0.1, 1, 'scale', 10, 100],
    'kernel': ['rbf','linear']
}

grid_search_svm, best_params_svm, best_score_svm = grid_search_hyperparameter_tuning(
    model=SVC(probability=True, random_state=42),
    param_grid=param_grid_svm,
    X_train=X_train,
    y_train=y_train
)

In [ ]:
# Convert results to DataFrame
cv_results = pd.DataFrame(grid_search_svm.cv_results_)

# Convert parameters to numeric type for correct sorting
cv_results["param_C"] = cv_results["param_C"].astype(float)
cv_results["param_gamma"] = cv_results["param_gamma"].astype(str)  # Keep gamma as string if 'scale' is used

# Use pivot_table instead of pivot to handle duplicate index-column pairs
df_heatmap = cv_results.pivot_table(index="param_C", columns="param_gamma", values="mean_test_score", aggfunc="mean")

# Sort the index for better visualization
df_heatmap = df_heatmap.sort_index(ascending=True)

# Plot heatmap
plt.figure(figsize=(14, 6))
sns.heatmap(df_heatmap, annot=True, cmap="coolwarm", fmt=".3f")
plt.title("AUC Scores for Different C and Gamma Values")
plt.xlabel("Gamma")
plt.ylabel("C")
plt.show()


In [ ]:
# Initialize the SVM classifier with the best hyperparameters
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)

# Train the SVM classifier on the training set
svm.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm.predict(X_test)
y_prob = svm.predict_proba(X_test)

# Evaluate AUC on the test set
test_auc = roc_auc_score(y_test, y_prob, multi_class='ovr')
print("\nAUC on the test set:", test_auc)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("\nAccuracy on the test set:", accuracy)

# Visualize the confusion matrix using heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_train), yticklabels=np.unique(y_train))
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Binarize the labels for multi-class ROC (One-vs-Rest)
y_test_binarized = label_binarize(y_test, classes=np.unique(y_test))

# Initialize a plot
plt.figure(figsize=(10, 8))

# Loop through each class and compute ROC curve and AUC
for i in range(len(np.unique(y_test))):
    fpr, tpr, _ = roc_curve(y_test_binarized[:, i], y_prob[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'Class {np.unique(y_test)[i]} (AUC = {roc_auc:.2f})')

# Plot settings
plt.plot([0, 1], [0, 1], 'k--', lw=2)  # Diagonal line (random classifier)
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Multi-class ROC Curve')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Reduce features to 30 dimensions using PCA
pca = PCA(n_components=0.8)  
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)  

print("Explained variance ratio:", pca.explained_variance_ratio_)
print("Reduced feature shape:", X_train_pca.shape)

In [ ]:
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
svm.fit(X_train_pca, y_train)

# Step 3: Evaluate the classifier
y_pred = svm.predict(X_test_pca)
y_prob = svm.predict_proba(X_test_pca)

# Step 4: Calculate metrics
test_auc = roc_auc_score(y_test, y_prob, multi_class='ovr')
print("\nAUC on the test set:", test_auc)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_train), yticklabels=np.unique(y_train))
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
results = test_feature_combinations(
    
    combinations_dict=feature_combinations,
    y=y,
    model=SVC(),  # Specify the SVM model
    model_name="SVM",  # Provide a name for display pu,rposes
    model_params={'kernel': 'rbf', 'C': 1, 'gamma': 'scale', 'probability': True, 'random_state': 42},  # Custom SVM parameters
    n_splits=5,  # Number of Stratified K-Folds
    normalize=True,  # Enable normalization of features
    apply_pca=True,  # Apply PCA during feature processing
    n_pca_components=0.9,  # Number of PCA components (explained variance ratio)
    overfit_threshold=0.15,  # Overfitting threshold
    use_gridsearch=False,  # Disable GridSearchCV for now
    hyperparameter_grid={
        'C': [0.1, 1, 10],
        'gamma': ['scale', 0.1, 0.01],
        'kernel': ['rbf']
    } if False else None,  # Use direct Boolean instead of undefined variable
    top_k_results=5,  # Display the top 5 combinations
    verbose=False  # Enable detailed output for each combination
)

In [ ]:
top_5_results_length_lim = [res for res in results if len(res['combination']) == 3 and res['overfitting_status'] == "✅ No Overfitting"]
top_5_results_length_lim.sort(key=lambda x: x['combined_score'], reverse=True)

print("\nTop 5 Non-Overfitted Combinations with Exactly 3 Features:")
for res in top_5_results_length_lim[:5]:  # Display the top 5
    print(f"Combination: {res['combination']}, AUC: {res['average_auc']:.4f}, "
          f"Val Accuracy: {res['average_val_accuracy']:.4f}, Overfitting: {res['overfitting_status']}")

In [ ]:
# Extract information from top 5 non-overfitted results
feature_names = ["_".join(res['combination']) for res in top_5_results_length_lim[:5]]
accuracies = [res['average_val_accuracy'] for res in top_5_results_length_lim[:5]]
auc_scores = [res['average_auc'] for res in top_5_results_length_lim[:5]]

# Create the figure and axis
fig, ax1 = plt.subplots(figsize=(12, 6))

# Use different colors for bars
colors = sns.color_palette("Blues", len(feature_names))

# Plot Accuracy (Bar chart)
ax1.bar(feature_names, accuracies, color=colors, alpha=0.7, label='Accuracy')
ax1.set_ylabel('Accuracy', color='b')
ax1.set_ylim(0.5, 1.0)  # Set limits to fit the data well

# Rotate X-axis labels for better readability
plt.xticks(rotation=30, ha="right", fontsize=10)

# Add grid lines for better visual separation
ax1.grid(axis='y', linestyle='--', alpha=0.5)

# Create a second y-axis for AUC scores
ax2 = ax1.twinx()

# Plot AUC as a red line with markers
ax2.plot(feature_names, auc_scores, color='r', marker='o', linestyle='-', linewidth=2, markersize=8, label='AUC')
ax2.set_ylabel('AUC Score', color='r')
ax2.set_ylim(0.5, 1.0)  # Set limits to fit the data well

# Formatting
plt.title("SVM Performance on Top 5 Feature Combinations", fontsize=14, fontweight='bold')

# Add legends
ax1.legend(loc='upper left', fontsize=10)
ax2.legend(loc='upper right', fontsize=10)

# Show the plot
plt.show()

In [ ]:
feature_selection = {
    'mfcc': True,
    'hist': True,
    'spectral_centroid': True,
    'spectral_contrast': True,
    'pitch_features': True,
    'zcr': True,
    'envelope': True,
    'hnr': True
}

selected_features, X_train, X_test, y_train, y_test, selected_features_names = load_and_split_features(
    loaded_data, feature_selection
)


In [ ]:
svm_results = kfold_cross_validation(
    features=selected_features,
    labels=y,
    selected_features_names=selected_features_names,
    model=SVC(probability=True),
    model_params={'kernel': 'rbf', 'C': 1.0, 'gamma': 0.01, 'random_state': 42},
    n_splits=5,
    preprocess_params={'normalize': True, 'apply_pca': False, 'n_pca_components': 0.9},
    overfit_threshold=0.1
)

---
---

# RF

In [ ]:
# Load saved features
loaded_data = np.load("features/extracted_features_multiple_test.npz")

# Define which features to include (set True to include, False to exclude)
feature_selection = {
    'mfcc': True,
    'hist': True,
    'spectral_centroid': True,
    'spectral_contrast': True,
    'pitch_features': True,
    'zcr': True,
    'envelope': True,
    'hnr': True
}

selected_features, X_train, X_test, y_train, y_test, selected_features_names = load_and_split_features(
    loaded_data, feature_selection
)


In [ ]:
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
y_prob_rf = rf.predict_proba(X_test)

In [ ]:
# AUC score
test_auc_rf = roc_auc_score(y_test, y_prob_rf, multi_class='ovr')

# Accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)

# Classification report
print("\nRandom Forest - AUC on the test set:", test_auc_rf)
print("\nRandom Forest - Accuracy on the test set:", accuracy_rf)
y_train_pred = rf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("\nTraining Accuracy:", train_accuracy)
print("\nRandom Forest - Classification Report:")
print(classification_report(y_test, y_pred_rf))

# Confusion matrix visualization
cm_rf = confusion_matrix(y_test, y_pred_rf)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_train), yticklabels=np.unique(y_train))
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Random Forest - Confusion Matrix')
plt.show()

In [ ]:
y_train_pred = rf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("\nTraining Accuracy:", train_accuracy)

In [ ]:
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

grid_search_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=5, scoring='roc_auc_ovr', n_jobs=-1, verbose=1)
grid_search_rf.fit(X_train, y_train)
print("\nBest Parameters for Random Forest:", grid_search_rf.best_params_)

In [ ]:
rf = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=10, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
y_prob_rf = rf.predict_proba(X_test)

In [ ]:
test_auc_rf = roc_auc_score(y_test, y_prob_rf, multi_class='ovr')
accuracy_rf = accuracy_score(y_test, y_pred_rf)

# Classification report
print("\nRandom Forest - AUC on the test set:", test_auc_rf)
print("\nRandom Forest - Accuracy on the test set:", accuracy_rf)
y_train_pred = rf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("\nTraining Accuracy:", train_accuracy)
print("\nRandom Forest - Classification Report:")
print(classification_report(y_test, y_pred_rf))

# Confusion matrix visualization
cm_rf = confusion_matrix(y_test, y_pred_rf)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_train), yticklabels=np.unique(y_train))
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Random Forest - Confusion Matrix')
plt.show()

In [ ]:
rf_results = test_feature_combinations(
    combinations_dict=feature_combinations,  # Your feature dictionary
    y=y,  # Your corresponding labels
    model=RandomForestClassifier(),  # RF model
    model_name="Random Forest",  # Model name for display purposes
    model_params={  # Default RF parameters (if grid search is off)
        'n_estimators': 100,
        'max_depth': 10,
        'min_samples_split': 5,
        'random_state': 42
    },
    n_splits=5,  # Number of Stratified K-Folds
    normalize=False,  # Disable normalization of features
    apply_pca=False,  # Disable PCA here for testing; set to True if needed
    n_pca_components=10,  # Number of PCA components to apply if needed
    overfit_threshold=0.2,  # Threshold to detect overfitting
    use_gridsearch=False,  # Disable or enable GridSearch
    hyperparameter_grid={  # Grid is always passed, but used only when use_gridsearch=True
        'n_estimators': [50, 100, 150],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10]
    },
    top_k_results=5,  # Display top 5 combinations
    verbose=True  # Enable detailed output for each combination
)

In [ ]:
feature_selection = {
    'mfcc': False,
    'hist': False,
    'spectral_centroid': True,
    'spectral_contrast': True,
    'pitch_features': True,
    'zcr': False,
    'envelope': False,
    'hnr': False
}
selected_features, X_train, X_test, y_train, y_test, selected_features_names = load_and_split_features(
    loaded_data, feature_selection
)


In [ ]:
rf_results = kfold_cross_validation(
    features=selected_features,
    labels=y,
    selected_features_names=selected_features_names,
    model=RandomForestClassifier(),
    model_params={
        'n_estimators': 100,
        'max_depth': 10,
        'min_samples_split': 5,
        'random_state': 42
    },
    n_splits=5,
    preprocess_params={'normalize': False, 'apply_pca': False, 'n_pca_components': 10},
    overfit_threshold=0.15
)

---
---

# KNN

In [ ]:
# Load saved features
loaded_data = np.load("features/extracted_features_multiple_test.npz")

# Define which features to include (set True to include, False to exclude)
feature_selection = {
    'mfcc': True,
    'hist': True,
    'spectral_centroid': True,
    'spectral_contrast': True,
    'pitch_features': True,
    'zcr': True,
    'envelope': True,
    'hnr': True
}

selected_features, X_train, X_test, y_train, y_test, selected_features_names = load_and_split_features(
    loaded_data, feature_selection
)

X_train, X_test = preprocess_features(
    X_train, X_test, normalize=True, apply_pca=False, n_pca_components=0.9, verbose=True
)


In [ ]:
# Initialize the KNN classifier with k=3
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Predictions on test set
y_pred = knn.predict(X_test)
y_prob = knn.predict_proba(X_test)

# Evaluation
test_accuracy = accuracy_score(y_test, y_pred)
test_auc = roc_auc_score(y_test, y_prob, multi_class='ovr')

# Predictions on train set for training accuracy
y_train_pred = knn.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)

# Print evaluation metrics
print(f"\nKNN Test Accuracy: {test_accuracy:.4f}")
print(f"KNN Test AUC: {test_auc:.4f}")
print(f"\nKNN Training Accuracy: {train_accuracy:.4f}")

print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Confusion matrix visualization
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('KNN Confusion Matrix')
plt.show()

In [ ]:
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

grid_search_knn, best_params_knn, best_score_knn = grid_search_hyperparameter_tuning(
    model=KNeighborsClassifier(),
    param_grid=param_grid_knn,
    X_train=X_train,
    y_train=y_train
)

In [ ]:
def test_feature_combinations(
    combinations_dict,
    y,
    model,
    model_name,
    model_params=None,
    n_splits=5,
    normalize=True,
    apply_pca=True,
    n_pca_components=0.9,
    overfit_threshold=0.1,
    hyperparameter_grid=None,
    use_gridsearch=False,
    top_k_results=5,
    verbose=False
):
    results = []

    # Generate all possible non-empty combinations of features
    all_combinations = [comb for i in range(1, len(combinations_dict) + 1) 
                        for comb in itertools.combinations(combinations_dict.keys(), i)]

    print(f"\n🔍 Testing {len(all_combinations)} feature combinations for {model_name}...\n")

    for combination in all_combinations:
        # Select features
        selected_features = np.hstack([combinations_dict[feature] for feature in combination])

        # Split the data
        X_train, X_test, y_train, y_test = train_test_split(
            selected_features, y, test_size=0.2, random_state=42, stratify=y
        )

        # Preprocess features (Normalize, PCA if applicable)
        X_train, X_test = preprocess_features(
            X_train, X_test, 
            normalize=normalize, 
            apply_pca=apply_pca, 
            n_pca_components=n_pca_components,
            verbose=False  # Silence preprocessing output during the loop
        )

        # Optionally perform GridSearch
        if use_gridsearch:
            if hyperparameter_grid:
                print(f"\nPerforming grid search for combination: {combination}...")
                grid_search, best_params, best_score = grid_search_hyperparameter_tuning(
                    model=model,
                    param_grid=hyperparameter_grid,
                    X_train=X_train,
                    y_train=y_train
                )
                model_params = best_params
                print(f"Grid search complete. Best parameters: {best_params}, Best AUC: {best_score:.4f}")
            else:
                raise ValueError("Grid search enabled, but no hyperparameter grid provided.")
        elif not model_params:
            raise ValueError("Provide `model_params` or enable grid search.")

        # Cross-validation using K-Fold
        train_accuracies, val_accuracies, auc_scores = [], [], []
        kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

        for train_idx, val_idx in kfold.split(X_train, y_train):
            fold_X_train, fold_X_val = X_train[train_idx], X_train[val_idx]
            fold_y_train, fold_y_val = y_train[train_idx], y_train[val_idx]

            # Initialize and train the model
            classifier = model.set_params(**model_params)
            classifier.fit(fold_X_train, fold_y_train)

            # Training and validation accuracy
            train_acc = accuracy_score(fold_y_train, classifier.predict(fold_X_train))
            val_acc = accuracy_score(fold_y_val, classifier.predict(fold_X_val))
            train_accuracies.append(train_acc)
            val_accuracies.append(val_acc)

            # Calculate AUC if available
            if hasattr(classifier, "predict_proba"):
                y_val_proba = classifier.predict_proba(fold_X_val)
                auc_scores.append(roc_auc_score(fold_y_val, y_val_proba, multi_class='ovr'))

        # Calculate averages
        avg_train_accuracy = np.mean(train_accuracies)
        avg_val_accuracy = np.mean(val_accuracies)
        avg_auc = np.mean(auc_scores) if auc_scores else None
        overfitting_gap = avg_train_accuracy - avg_val_accuracy
        overfitting_status = "⚠️ Overfitting Risk" if overfitting_gap > overfit_threshold else "✅ No Overfitting"

        # Calculate combined score: 50% AUC + 50% validation accuracy
        combined_score = 0.5 * (avg_auc if avg_auc is not None else 0) + 0.5 * avg_val_accuracy

        # Store results
        results.append({
            'combination': combination,
            'average_auc': avg_auc,
            'average_train_accuracy': avg_train_accuracy,
            'average_val_accuracy': avg_val_accuracy,
            'combined_score': combined_score,
            'overfitting_status': overfitting_status,
            'overfitting_gap': overfitting_gap
        })

        if verbose:
            print(f"Combination: {combination}, "
                  f"AUC: {f'{avg_auc:.4f}' if avg_auc is not None else 'N/A'}, "
                  f"Train Acc: {avg_train_accuracy:.4f}, "
                  f"Val Acc: {avg_val_accuracy:.4f}, "
                  f"Overfitting: {overfitting_status}")

    # Filter and sort by combined score
    sorted_results = sorted(results, key=lambda x: x['combined_score'], reverse=True)

    # Display top-k results
    print(f"\n📊 Top {top_k_results} Results (sorted by Combined Score):")
    for i, res in enumerate(sorted_results[:top_k_results], start=1):
        print(f"  {i}. Combination: {res['combination']}, "
              f"Combined Score: {res['combined_score']:.4f}, "
              f"Test Acc: {res['average_val_accuracy']:.4f}, "
              f"Train Acc: {res['average_train_accuracy']:.4f}, "
              f"AUC: {f'{res['average_auc']:.4f}' if res['average_auc'] is not None else 'N/A'}, "
              f"Train-Test Gap: {res['overfitting_gap']:.4f}, "
              f"Overfitting Status: {res['overfitting_status']}")

    return sorted_results

In [ ]:
knn_results = test_feature_combinations(
    combinations_dict=feature_combinations,  # Your feature dictionary
    y=y,  # Your corresponding labels
    model=KNeighborsClassifier(),  # KNN model
    model_name="KNN",  # Model name for display purposes
    model_params={
        'n_neighbors': 7,
        'weights': 'uniform',
        'metric': 'euclidean'
    },  # KNN parameters
    n_splits=5,  # Number of Stratified K-Folds
    normalize=True,  # Enable normalization of features
    apply_pca=False,  # Disable PCA for now
    n_pca_components=10,  # Number of PCA components if apply_pca=True
    overfit_threshold=0.1,  # Threshold to detect overfitting
    use_gridsearch=False,  # Disable grid search in this example
    top_k_results=5,  # Show top 5 combinations
    verbose=True  # printing all combinations during the loop
)

In [ ]:
feature_selection = {
    'mfcc': True,
    'hist': False,
    'spectral_centroid': True,
    'spectral_contrast': True,
    'pitch_features': True,
    'zcr': False,
    'envelope': True,
    'hnr': True
}

selected_features, X_train, X_test, y_train, y_test, selected_features_names = load_and_split_features(
    loaded_data, feature_selection
)

X_train, X_test = preprocess_features(
    X_train, X_test, normalize=True, apply_pca=False, n_pca_components=0.9
)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7, weights='uniform', metric='euclidean')
knn.fit(X_train, y_train)

# Predictions on test set
y_pred = knn.predict(X_test)
y_prob = knn.predict_proba(X_test)

# Evaluation
test_accuracy = accuracy_score(y_test, y_pred)
test_auc = roc_auc_score(y_test, y_prob, multi_class='ovr')

# Predictions on train set for training accuracy
y_train_pred = knn.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)

# Print evaluation metrics
print(f"\nKNN Test Accuracy: {test_accuracy:.4f}")
print(f"KNN Test AUC: {test_auc:.4f}")
print(f"\nKNN Training Accuracy: {train_accuracy:.4f}")

print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Confusion matrix visualization
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('KNN Confusion Matrix')
plt.show()

In [ ]:
feature_selection = {
    'mfcc': True,
    'hist': False,
    'spectral_centroid': True,
    'spectral_contrast': True,
    'pitch_features': True,
    'zcr': False,
    'envelope': True,
    'hnr': True
}

# selected_features, selected_features_names = get_selected_features(feature_combinations, feature_selection)

selected_features, X_train, X_test, y_train, y_test, selected_features_names = load_and_split_features(
    loaded_data, feature_selection
)

In [ ]:
knn_results = kfold_cross_validation(
    features=selected_features,
    labels=y,
    selected_features_names=selected_features_names,
    model=KNeighborsClassifier(),  # Initialize the KNN model
    model_params={
        'n_neighbors': 7,
        'weights': 'uniform',
        'metric': 'euclidean'
    },
    n_splits=5,
    preprocess_params={'normalize': True, 'apply_pca': False, 'n_pca_components': 0.8},
    overfit_threshold=0.15
)

---
---

# K-Means

In [ ]:
from scipy.stats import mode

# 1️⃣ Step 1: Feature Selection
feature_selection = {
    'mfcc': True,
    'hist': False,
    'spectral_centroid': True,
    'spectral_contrast': True,
    'pitch_features': True,
    'zcr': True,
    'envelope': True,
    'hnr': True
}

# Load selected features
selected_features, X_train, X_test, y_train, y_test, selected_features_names = load_and_split_features(
    loaded_data, feature_selection
)

# 2️⃣ Step 2: Normalize the Features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)  # Use the same scaler for the test set



In [ ]:
# 3️⃣ Step 3: Apply PCA for Dimensionality Reduction
pca = PCA(n_components=0.95)  # Keep 95% variance
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

print("Number of PCA Components Used:", pca.n_components_)
print("Explained Variance:", np.sum(pca.explained_variance_ratio_))

# 4️⃣ Step 4: Find the Optimal Number of Clusters using the Elbow Method
inertia = []
sil_scores = []  # ✅ FIXED: Define silhouette scores list

K_range = range(2, 20)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=20)
    labels = kmeans.fit_predict(X_train_pca)
    
    inertia.append(kmeans.inertia_)
    sil_scores.append(silhouette_score(X_train_pca, labels))  # ✅ FIXED: Compute Silhouette Score

# ✅ Plot Elbow Curve
plt.figure(figsize=(8,5))
plt.plot(K_range, inertia, marker='o')
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Inertia")
plt.title("Elbow Method for Optimal k")
plt.show()

# ✅ Plot Silhouette Scores
plt.figure(figsize=(8,5))
plt.plot(K_range, sil_scores, marker='o', label='Silhouette Score', color='red')
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Score for Different k Values")
plt.show()

# 5️⃣ Step 5: Perform K-Means Clustering with k=10 (Known Value)
best_k = 10  # Since we already know the correct number of clusters

#kmeans = KMeans(n_clusters=best_k, init='k-means++', random_state=42, n_init=20, max_iter=5000) 
kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=30)
cluster_labels = kmeans.fit_predict(X_train_pca)

# 6️⃣ Step 6: Evaluate Clustering Performance using Silhouette Score
silhouette = silhouette_score(X_train_pca, cluster_labels)
print(f"Silhouette Score for k=10: {silhouette:.4f}")

# 7️⃣ Step 7: Visualize Clusters using t-SNE
X_embedded = TSNE(n_components=2, perplexity=15, random_state=42).fit_transform(X_train_pca)  # ✅ FIXED Perplexity

plt.figure(figsize=(8,6))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=cluster_labels, cmap='tab10', alpha=0.7)
plt.colorbar()
plt.title("t-SNE Visualization of Clusters")
plt.show()

# 8️⃣ Step 8: Evaluate K-Means with Adjusted Rand Index (ARI)
ari_score = adjusted_rand_score(y_train, cluster_labels)
print(f"Adjusted Rand Index (ARI) for K-Means: {ari_score:.4f}")


# 🔟 Step 9: Assign Labels to Clusters using Majority Voting
def assign_labels_majority(y_true, cluster_labels):
    assigned_labels = np.zeros_like(cluster_labels)

    for cluster in np.unique(cluster_labels):
        mask = (cluster_labels == cluster)
        true_labels = y_true[mask]  # Get true labels in this cluster
        
        if len(true_labels) == 0:  # Edge case: No data points in cluster
            continue 
        
        if len(true_labels) > 0:
            majority_label = mode(true_labels, keepdims=True).mode[0]
        else:
            majority_label = -1  # Assign an unused label if cluster is empty

        
        assigned_labels[mask] = majority_label  # Assign majority label to all cluster members

    return assigned_labels

y_pred = assign_labels_majority(y_train, cluster_labels)

# 1️⃣1️⃣ Step 11: Confusion Matrix
cm = confusion_matrix(y_train, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Cluster')
plt.ylabel('True Label')
plt.title('Confusion Matrix: True Labels vs. Cluster Assignments')
plt.show()


In [ ]:
unique, counts = np.unique(cluster_labels, return_counts=True)
cluster_distribution = dict(zip(unique, counts))

print("\n🔍 Cluster Size Distribution:")
for cluster, count in sorted(cluster_distribution.items()):
    print(f"Cluster {cluster}: {count} samples")


In [ ]:
for cluster in np.unique(cluster_labels):
    assigned_class = mode(y_train[cluster_labels == cluster], keepdims=True).mode[0]
    print(f"Cluster {cluster} is mostly class {assigned_class}")


In [ ]:
cluster_class_df = pd.DataFrame({'Cluster': cluster_labels, 'True Class': y_train})
cluster_counts = cluster_class_df.groupby(['Cluster', 'True Class']).size().unstack(fill_value=0)

cluster_counts.plot(kind='bar', stacked=True, figsize=(10,6), colormap='tab10')
plt.xlabel("Cluster")
plt.ylabel("Number of Samples")
plt.title("Class Distribution in Each Cluster")
plt.legend(title="True Class")
plt.show()
